# Bi-linear interaction model with group-by augmentations

## Prepare dataset for training

Following the same steps as in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb), we first load splitted dataset generated in [notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/dataset_preprocessing/movielens%20with%20imdb.ipynb)

In [1]:
DATASET = 'rees_ecommerce'

In [2]:
from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

Then we parse features' names to obtain a list of offer features and a list of user features (aggregated history up to chosen date)

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
technical_columns = ['user_id', 'date']
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + technical_columns, all_columns))

### Rebatching datasets

Splitting dataset into smaller batches in the same way as described in [the training for Movielens/IMDB dataset](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb)

In [4]:
%%time

from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=5040, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=5040, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

CPU times: user 21.6 s, sys: 861 ms, total: 22.5 s
Wall time: 11.4 s


In [5]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))

## Define the model

First we need to get number of different modalities inputs can take from saved vectorizers (it will be used in embeddings layer definition):

In [6]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [7]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]

### Model architecture

In [8]:
import tensorflow as tf

Now we can assemble all these layers into final model. Note that offer compression weights and interaction kernels are shared between different augmentations we generate.

<img src="resources/group_by_augmentations_model.png" alt="model" width="800" />

### Model parameters

In [9]:
EMBEDDING_DIM = 100
L1_COEFF = 2e-7
DROPOUT = 0.1
NB_AUGMENTATIONS = 3
AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION = 2
USER_META_FEATURES = 5
OFFER_META_FEATURES = 3

def REGULARIZER():
    return {'class_name': 'L1L2', 'config': {'l1': L1_COEFF, 'l2': 0.}}

def OUTPUT_DNN():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(50,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(1,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
    ], name='output_Dnn')

EPOCHS = 8

NUMBER_OF_NEGATIVES = 4
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
AUC_METRIC = tf.keras.metrics.AUC(from_logits=True)

import tensorflow_addons as tfa
OPTIMIZER = tfa.optimizers.AdamW(weight_decay=4e-8, learning_rate=0.0008)

### Embeddings

We will define embeddings with the same `WeightedEmbeddings` layer described in [the training of a simple model](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/movielens%20simple%20model.ipynb) with one addition:
* for offer features when aggregating a list of embedding vectors, we will also calculate variance and not only mean vector

It is easy to do in the same sparse-dense matrix multiplication operation as mean calculation (we get second moment and then calculate variance from it).

In [10]:
from layers import get_input_layer, WeightedEmbeddings
from utils import WEIGHT_SUFFIX

inputs = {}
embedded_user_features, embedded_offer_features, variance_offer_features = {}, {}, {}
for feature in user_features:
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embedded_user_features[feature] = emb_layer(inputs[feature])
for feature in offer_features:
    # for offer features we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    inputs[f'{feature}_weight'] = get_input_layer(f'{feature}_weight', tf.float32)
    inputs[feature] = get_input_layer(feature)
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER(),
                                   calculate_variance=True)
    embedded_offer_features[feature], variance_offer_features[feature] =\
        emb_layer(inputs[feature], inputs[f'{feature}_weight'])

### Combining everything into model

Now we can define described model architecture on the top of embeddings.

In [11]:
user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
offer_stacked = tf.stack(list(embedded_offer_features.values()), axis=1)
offer_variance = tf.stack(list(variance_offer_features.values()), axis=1)
stacked_raw_offer_attrs = tf.stack([tf.cast(inp.values, tf.int32) for feature, inp in inputs.items()
                                    if feature in offer_features], axis=1)

In [12]:
from layers import KeyGenerator, GroupBy
from layers import UserFeaturesCompressor
from layers import OfferFeaturesCompressor

from layers import MaskNet

from layers import BiLinearInteraction


group_by = GroupBy(name='group_by')
key_generator = KeyGenerator(number_of_offer_attributes=len(offer_features),
                             average_number_of_attributes_in_key=AVERAGE_NUMBER_OF_FEATURES_IN_AUGMENTATION,
                             name='grp_key_generator')

user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                         name='user_compressor')(user_stacked)
offer_features_compressor = OfferFeaturesCompressor(OFFER_META_FEATURES, DROPOUT, name='offer_compressor')
mask_net = MaskNet(OFFER_META_FEATURES, DROPOUT, name='mask_generation')
apply_mask = tf.keras.layers.Multiply(name='apply_mask')
bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                            initializer='random_normal', regularizer=REGULARIZER(),
                                            name='interaction')
output_dnn = OUTPUT_DNN()

augmentation_predictions = []
for i in range(NB_AUGMENTATIONS):
    group_by_key = key_generator(stacked_raw_offer_attrs)
    mean_offer_emb, variance_offer_emb = group_by(group_by_key, offer_stacked)
    compressed_offer_embeddings = offer_features_compressor([mean_offer_emb, variance_offer_emb])
    mask = mask_net([mean_offer_emb, variance_offer_emb])
    masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])
    _output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))
    augmentation_predictions.append(_output)
output = tf.concat(augmentation_predictions, axis=1)

And for evaluation we don't need to create augmentations, we need just to take offer features' mean and variance coming from inputs.

In [13]:
compressed_offer_embeddings = offer_features_compressor([offer_stacked, offer_variance])
mask = mask_net([offer_stacked, offer_variance])
masked_offer_embeddings = apply_mask([compressed_offer_embeddings, mask])

eval_output = output_dnn(bi_linear_interaction([user_compressed, masked_offer_embeddings], generate_negatives=True))

In [14]:
from utils import BroadcastLoss, BroadcastMetric

model = tf.keras.Model(inputs, output, name='group_by_augmentations')
model.compile(optimizer=OPTIMIZER,
              loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
              metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])

eval_model = tf.keras.Model(inputs, eval_output, name='group_by_augmentations_eval')

### Training

In [15]:
model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/8


/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['date', 'user_id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/group_by_augmentations/group_by/moments_2/Reshape_4:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/group_by_augmentations/group_by/moments_2/Reshape_3:0", shape=(None, 6, 100), dtype=float32), dense_shape=Tensor("gradient_tape/group_by_augmentations/group_by/moments_2/Cast:0", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/jupyter/.local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse

679/679 [==============================] - 329s 417ms/step - loss: 0.4712 - auc: 0.6883 - val_loss: 0.4472 - val_auc: 0.7193
Epoch 2/8
679/679 [==============================] - 259s 380ms/step - loss: 0.4459 - auc: 0.7191 - val_loss: 0.4450 - val_auc: 0.7210
Epoch 3/8
679/679 [==============================] - 252s 370ms/step - loss: 0.4414 - auc: 0.7267 - val_loss: 0.4438 - val_auc: 0.7236
Epoch 4/8
679/679 [==============================] - 249s 366ms/step - loss: 0.4397 - auc: 0.7300 - val_loss: 0.4434 - val_auc: 0.7253
Epoch 5/8
679/679 [==============================] - 252s 370ms/step - loss: 0.4392 - auc: 0.7311 - val_loss: 0.4384 - val_auc: 0.7335
Epoch 6/8
679/679 [==============================] - 256s 377ms/step - loss: 0.4379 - auc: 0.7341 - val_loss: 0.4446 - val_auc: 0.7255
Epoch 7/8
679/679 [==============================] - 258s 380ms/step - loss: 0.4369 - auc: 0.7364 - val_loss: 0.4419 - val_auc: 0.7295
Epoch 8/8
679/679 [==============================] - 255s 375ms/s

## Single task models benchmark

Using same approach as in [the simple model notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/training/rees%20simple%20model.ipynb) we will look at performance gap between the model with group by augmentations against set of models specialized on tasks corresponding to one offer feature at time. We won't use augmentations in those baseline models, because they will be already aligned with offer we will use in evaluation afterwards. To illustrate importance of augmentations we will also train single model without group_by augmentations.

In [16]:
# offer columns we want to evaluate, specific to dataset we test
TASKS = ['product_id', 'category1', 'category2', 'category3', 'brand', 'priceCluster']

In [19]:
def bi_linear_interaction_model(offer_features, name='bi_linear_model'):
    # user_features, vocabulary_sizes, EMBEDDING_DIM, REGULARIZER, OPTIMIZER,
    # OUTPUT_DNN, LOSS, NUMBER_OF_NEGATIVES
    # come from global scope, but can be passed as params instead
    inputs = {}
    embedded_user_features, embedded_offer_features = {}, {}
    for feature in user_features:
        inputs[feature] = get_input_layer(feature)
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embedded_user_features[feature] = emb_layer(inputs[feature])

    # for offer feature we need weights:
    # with dummy weights during training, and the ones used for a feature's averaging at inference time
    for feature in offer_features:
        inputs[feature] = get_input_layer(feature)
        inputs[f'{feature}_weight'] = get_input_layer(f'{feature}_weight', tf.float32)
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embedded_offer_features[feature] = emb_layer(inputs[feature], inputs[f'{feature}_weight'])
    
    user_stacked = tf.stack(list(embedded_user_features.values()), axis=1)
    offer_stacked = tf.stack(list(embedded_offer_features.values()), axis=1)
    
    user_compressed = UserFeaturesCompressor(USER_META_FEATURES, DROPOUT,
                                             name='user_compressor')(user_stacked)
    
    output_dnn = OUTPUT_DNN()
    bi_linear_interaction = BiLinearInteraction(number_of_negatives=NUMBER_OF_NEGATIVES, dropout_rate=DROPOUT,
                                                initializer='random_normal', regularizer=REGULARIZER(),
                                                name='interaction')
    
    output = output_dnn(bi_linear_interaction([user_compressed, offer_stacked], generate_negatives=True))

    model = tf.keras.Model(inputs, output, name=name)
    model.compile(optimizer=OPTIMIZER,
                  loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
                  metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])
    
    return model

In [20]:
model_wo_augmentations = bi_linear_interaction_model(offer_features, name='model_wo_augm')
model_wo_augmentations.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/8
679/679 [==============================] - 200s 255ms/step - loss: 0.4385 - auc: 0.7385 - val_loss: 0.4212 - val_auc: 0.7628
Epoch 2/8
679/679 [==============================] - 157s 230ms/step - loss: 0.4162 - auc: 0.7688 - val_loss: 0.4175 - val_auc: 0.7674
Epoch 3/8
679/679 [==============================] - 156s 229ms/step - loss: 0.4121 - auc: 0.7745 - val_loss: 0.4165 - val_auc: 0.7684
Epoch 4/8
679/679 [==============================] - 158s 232ms/step - loss: 0.4098 - auc: 0.7789 - val_loss: 0.4178 - val_auc: 0.7687
Epoch 5/8
679/679 [==============================] - 158s 232ms/step - loss: 0.4083 - auc: 0.7830 - val_loss: 0.4192 - val_auc: 0.7688
Epoch 6/8
679/679 [==============================] - 157s 231ms/step - loss: 0.4068 - auc: 0.7875 - val_loss: 0.4231 - val_auc: 0.7677
Epoch 7/8
679/679 [==============================] - 156s 230ms/step - loss: 0.4055 - auc: 0.7923 - val_loss: 0.4265 - val_auc: 0.7666
Epoch 8/8
679/679 [==============================] - 15

In [21]:
mono_feature_models = {}
for task_offer_feature in TASKS:
    mono_feature_models[task_offer_feature] =\
        bi_linear_interaction_model([task_offer_feature], name=f'{task_offer_feature}_model')
    mono_feature_models[task_offer_feature].fit(datasets['train'],
                                                epochs=EPOCHS,
                                                validation_data=datasets['val'])

Epoch 1/8


/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'category2', 'date', 'user_id', 'brand', 'priceCluster', 'category1', 'category3_weight', 'category2_weight', 'brand_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 156s 199ms/step - loss: 0.4527 - auc: 0.7337 - val_loss: 0.4383 - val_auc: 0.7497
Epoch 2/8
679/679 [==============================] - 125s 184ms/step - loss: 0.4286 - auc: 0.7646 - val_loss: 0.4347 - val_auc: 0.7550
Epoch 3/8
679/679 [==============================] - 125s 183ms/step - loss: 0.4221 - auc: 0.7742 - val_loss: 0.4355 - val_auc: 0.7563
Epoch 4/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4184 - auc: 0.7807 - val_loss: 0.4377 - val_auc: 0.7557
Epoch 5/8
679/679 [==============================] - 126s 185ms/step - loss: 0.4153 - auc: 0.7861 - val_loss: 0.4387 - val_auc: 0.7558
Epoch 6/8
679/679 [==============================] - 128s 188ms/step - loss: 0.4130 - auc: 0.7905 - val_loss: 0.4417 - val_auc: 0.7540
Epoch 7/8
679/679 [==============================] - 140s 206ms/step - loss: 0.4112 - auc: 0.7943 - val_loss: 0.4452 - val_auc: 0.7527
Epoch 8/8
679/679 [==============================] - 135s 198ms/s

/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'category2', 'date', 'user_id', 'brand', 'product_id', 'priceCluster', 'category3_weight', 'category2_weight', 'brand_weight', 'product_id_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 169s 211ms/step - loss: 0.4749 - auc: 0.6775 - val_loss: 0.4697 - val_auc: 0.6725
Epoch 2/8
679/679 [==============================] - 123s 181ms/step - loss: 0.4690 - auc: 0.6729 - val_loss: 0.4692 - val_auc: 0.6731
Epoch 3/8
679/679 [==============================] - 123s 181ms/step - loss: 0.4683 - auc: 0.6742 - val_loss: 0.4687 - val_auc: 0.6734
Epoch 4/8
679/679 [==============================] - 123s 181ms/step - loss: 0.4677 - auc: 0.6755 - val_loss: 0.4686 - val_auc: 0.6736
Epoch 5/8
679/679 [==============================] - 123s 181ms/step - loss: 0.4672 - auc: 0.6769 - val_loss: 0.4690 - val_auc: 0.6733
Epoch 6/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4666 - auc: 0.6784 - val_loss: 0.4691 - val_auc: 0.6733
Epoch 7/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4660 - auc: 0.6802 - val_loss: 0.4696 - val_auc: 0.6727
Epoch 8/8
679/679 [==============================] - 123s 181ms/s

...

679/679 [==============================] - 153s 196ms/step - loss: 0.4701 - auc: 0.6743 - val_loss: 0.4628 - val_auc: 0.6817
Epoch 2/8
679/679 [==============================] - 126s 185ms/step - loss: 0.4621 - auc: 0.6827 - val_loss: 0.4622 - val_auc: 0.6830
Epoch 3/8
679/679 [==============================] - 127s 187ms/step - loss: 0.4609 - auc: 0.6849 - val_loss: 0.4627 - val_auc: 0.6834
Epoch 4/8
679/679 [==============================] - 127s 186ms/step - loss: 0.4603 - auc: 0.6863 - val_loss: 0.4620 - val_auc: 0.6838
Epoch 5/8
679/679 [==============================] - 126s 185ms/step - loss: 0.4598 - auc: 0.6877 - val_loss: 0.4625 - val_auc: 0.6840
Epoch 6/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4592 - auc: 0.6893 - val_loss: 0.4624 - val_auc: 0.6836
Epoch 7/8
679/679 [==============================] - 122s 179ms/step - loss: 0.4587 - auc: 0.6911 - val_loss: 0.4629 - val_auc: 0.6830
Epoch 8/8
679/679 [==============================] - 121s 178ms/s

/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'category2', 'date', 'user_id', 'product_id', 'priceCluster', 'category1', 'category3_weight', 'category2_weight', 'product_id_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 153s 194ms/step - loss: 0.4518 - auc: 0.7158 - val_loss: 0.4426 - val_auc: 0.7311
Epoch 2/8
679/679 [==============================] - 122s 179ms/step - loss: 0.4402 - auc: 0.7352 - val_loss: 0.4410 - val_auc: 0.7334
Epoch 3/8
679/679 [==============================] - 120s 176ms/step - loss: 0.4380 - auc: 0.7385 - val_loss: 0.4405 - val_auc: 0.7340
Epoch 4/8
679/679 [==============================] - 119s 175ms/step - loss: 0.4369 - auc: 0.7406 - val_loss: 0.4406 - val_auc: 0.7343
Epoch 5/8
679/679 [==============================] - 120s 177ms/step - loss: 0.4360 - auc: 0.7425 - val_loss: 0.4421 - val_auc: 0.7340
Epoch 6/8
679/679 [==============================] - 120s 176ms/step - loss: 0.4352 - auc: 0.7445 - val_loss: 0.4424 - val_auc: 0.7336
Epoch 7/8
679/679 [==============================] - 120s 176ms/step - loss: 0.4344 - auc: 0.7466 - val_loss: 0.4430 - val_auc: 0.7332
Epoch 8/8
679/679 [==============================] - 120s 177ms/s

/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'category2', 'date', 'user_id', 'brand', 'product_id', 'category1', 'category3_weight', 'category2_weight', 'brand_weight', 'product_id_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


679/679 [==============================] - 163s 205ms/step - loss: 0.4643 - auc: 0.6904 - val_loss: 0.4542 - val_auc: 0.6978
Epoch 2/8
679/679 [==============================] - 130s 191ms/step - loss: 0.4524 - auc: 0.7009 - val_loss: 0.4526 - val_auc: 0.7006
Epoch 3/8
679/679 [==============================] - 126s 185ms/step - loss: 0.4507 - auc: 0.7045 - val_loss: 0.4527 - val_auc: 0.7022
Epoch 4/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4500 - auc: 0.7066 - val_loss: 0.4525 - val_auc: 0.7026
Epoch 5/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4494 - auc: 0.7085 - val_loss: 0.4532 - val_auc: 0.7027
Epoch 6/8
679/679 [==============================] - 127s 187ms/step - loss: 0.4489 - auc: 0.7105 - val_loss: 0.4535 - val_auc: 0.7029
Epoch 7/8
679/679 [==============================] - 124s 182ms/step - loss: 0.4485 - auc: 0.7122 - val_loss: 0.4540 - val_auc: 0.7022
Epoch 8/8
679/679 [==============================] - 124s 183ms/s

## Evaluation

In [22]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

CPU times: user 3min 11s, sys: 16.3 s, total: 3min 27s
Wall time: 3min


In [23]:
%%time
from collections import defaultdict
from utils import evaluate_model

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    kw = {'single_task_feature': task_offer_feature, 'test_datasets': test_datasets,
          'number_of_negatives': NUMBER_OF_NEGATIVES, 'inverse_lookups': inverse_lookups}
    for model_name in TASKS:
        aucs[task_offer_feature][f'MONO:{model_name}'] = evaluate_model(mono_feature_models[model_name], **kw)
    aucs[task_offer_feature]['group_by augmentations'] = evaluate_model(eval_model, **kw)
    aucs[task_offer_feature]['without augmentations'] = evaluate_model(model_wo_augmentations, **kw)

/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'category2', 'date', 'user_id', 'brand', 'product_id', 'priceCluster', 'category3_weight', 'category2_weight', 'brand_weight', 'product_id_weight', 'priceCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category3', 'date', 'user_id', 'brand', 'product_id', 'priceCluster', 'category1', 'category3_weight', 'brand_weight', 'product_id_weight', 'priceCluster_weight', 'category1_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/home/jupyter/.local/lib/python3.9/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['category2', 'date'

CPU times: user 3h 26min 34s, sys: 28min 3s, total: 3h 54min 37s
Wall time: 57min 25s


In [24]:
from utils import save_metrics
save_metrics(aucs, DATASET, 'group_by_augmentations')

## Aggregating results

### Popular offers

First we look at well sampled values of each offer, with number of occurrences in test dataset higher than 200:

In [25]:
import pandas as pd
from utils import wAUC

results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name], cutoff_low=200)
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name, 'model': model_name}).to_frame().T],
                            ignore_index=True)

In [36]:
pd.pivot_table(results, 'wAUC', 'model', 'offers')\
    .rename(columns={'priceCluster': 'price'}, index={'MONO:priceCluster': 'MONO:price'})\
    .iloc[[6, 7, 5, 3, 0, 2, 1, 4]][['product_id', 'category3', 'brand', 'category2', 'category1', 'price']]\
    .style.background_gradient(cmap='coolwarm').format(precision=3)

offers,product_id,category3,brand,category2,category1,price
model,,,,,,
group_by augmentations,0.771,0.739,0.757,0.738,0.728,0.694
without augmentations,0.770,0.638,0.711,0.627,0.595,0.611
MONO:product_id,0.772,0.638,0.710,0.633,0.608,0.621
MONO:category3,0.660,0.745,0.654,0.710,0.690,0.566
MONO:brand,0.726,0.631,0.755,0.628,0.604,0.589
MONO:category2,0.667,0.709,0.664,0.742,0.721,0.561
MONO:category1,0.662,0.709,0.655,0.728,0.734,0.569
MONO:price,0.727,0.572,0.635,0.568,0.557,0.704


### Cold-start

Here we take a look on values with relatively small number of associated events (but not zero to have defined AUC: between 10 and 200)

In [38]:
results = pd.DataFrame()
for task_name in ['product_id', 'brand', 'category3']:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name], cutoff_low=10, cutoff_high=200)
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name, 'model': model_name}).to_frame().T],
                            ignore_index=True)

In [40]:
pd.pivot_table(results, 'wAUC', 'model', 'offers')\
    .rename(columns={'priceCluster': 'price'}, index={'MONO:priceCluster': 'MONO:price'})\
    .iloc[[6, 7, 5, 3, 0]][['product_id', 'category3', 'brand']]\
    .style.background_gradient(cmap='coolwarm').format(precision=3)

offers,product_id,category3,brand
model,,,
group_by augmentations,0.769,0.772,0.776
without augmentations,0.758,0.759,0.769
MONO:product_id,0.753,0.746,0.750
MONO:category3,0.697,0.767,0.710
MONO:brand,0.731,0.744,0.768
